In [24]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
appended_data = []
for year in range(1985, 2020):
    result = pd.DataFrame()
    url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(year)
    url2 = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year)
    url3 = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    
    
    #**********************EXTRACT DATA FROM FIRST URL**********************
    temp = pd.read_html(url, header=[1])
    temp[0].dropna(axis=1,inplace=True)
    temp = temp[0][0:10]
    temp.insert(1, 'Year', year)
    temp.insert(2, 'MVP', [1,0,0,0,0,0,0,0,0,0])
    temp.columns = temp.columns.str.strip()
    temp = temp.round(4)
    #**********************EXTRACT DATA FROM SECOND URL**********************
    temp2 = pd.read_html(url2)
    temp2 = temp2[0].dropna(axis=1,how='all')
    temp2['Player'] = temp2['Player'].str.replace('*','')
    temp2.columns = temp2.columns.str.strip()
    temp2 = temp2[temp2.Pos != 'Pos'] #get rid of random header rows
    temp2 = temp2.round(4)
    #Find intersecting columns
    cols = list(np.intersect1d(temp.columns, temp2.columns))
    #Remove cols temporarily that don't involve numbers because casting
    #will add them back later when doing 'on = cols'
    cols.remove('Player')
    cols.remove('Tm')
    #need to convert cols to float
    temp[cols] = temp[cols].astype(float)
    temp2[cols] = temp2[cols].astype(float) 
    
    #add qualitative analysis back for on = within merge function
    cols.append('Player')
    cols.append('Tm')
    
    #MERGE TEMP AND TEMP2
    result = temp.merge(temp2, how='inner', on=cols)
    if (year == 1985):
        result.to_csv('csv/TEMP1_2.csv')
    #**********************EXTRACT DATA FROM THIRD URL**********************
    temp3 = pd.read_html(url3)
    temp3 = temp3[0].dropna(axis=1, how='all')
    temp3 = temp3[temp3.Pos != 'Pos']
    temp3['Player'] = temp3['Player'].str.replace('*','')
    temp3 = temp3.drop('Rk', 1)
    temp3.columns = temp3.columns.str.strip()
    temp3 = temp3.rename(columns={"MP": "TOT_MP"})
    temp3 = temp3.round(4)
    #Find intersecting columns
    cols = list(np.intersect1d(result.columns, temp3.columns))

    #Remove cols temporarily that don't involve numbers because casting
    #will add them back later when doing 'on = cols'
    cols.remove('Player')
    cols.remove('Tm')
    cols.remove('Pos')
    #need to convert cols to float
    temp3[cols] = temp3[cols].astype(float) 
    temp3.to_csv('csv/temp3.csv')
    #add qualitative analysis back for on = within merge function
    cols.append('Player')
    cols.append('Tm')
    cols.append('Pos')
    result = result.merge(temp3, how='inner', on=cols)
    #add to list of data
    appended_data.append(result)
df = pd.concat(appended_data)
#data frame complete

In [27]:
 saved_df = df #saves df from cell 1 into another df to prevent having to constantly run 

In [28]:
df = saved_df #move dataframe over 

df = saved_df

In [29]:
import requests
import re
from bs4 import BeautifulSoup


df.count()
df['PRE_ODDS'] = 0
#This only works for [1985,2019]. For 2020, we will have to do something else and make a current csv to grab 
#data for games played to most recent date

modify = df.reset_index(drop=True)
modify.at[245, 'Tm'] = 'DEN'

df = modify
for i in range(len(df.index)):
    team = df.iloc[i]['Tm']
    year = df.iloc[i]['Year']
    odds_url = "https://www.basketball-reference.com/teams/{}/{}.html".format(team,year)
    #print(odds_url)
    html = requests.get(odds_url).text
    soup = BeautifulSoup(html, "lxml")
    res = soup.find('a', {"href": "/leagues/NBA_{}_preseason_odds.html".format(year)} ).text
    #df['PRE_ODDS'][i] = re.findall(r'\d+', res)[0]
    df.at[i, 'PRE_ODDS'] = re.findall(r'\-?\d+', res)[0]
#display(df)
df.to_csv('csv/historical_mvp_data.csv')

In [33]:

df.count()

Rank        350
Year        350
MVP         350
Player      350
Age         350
Tm          350
First       350
Pts Won     350
Pts Max     350
Share       350
G           350
MP          350
PTS         350
TRB         350
AST         350
STL         350
BLK         350
FG%         350
FT%         350
WS          350
WS/48       350
Rk          350
Pos         350
GS          350
FG          350
FGA         350
3P          350
3PA         350
3P%         344
2P          350
2PA         350
2P%         350
eFG%        350
FT          350
FTA         350
ORB         350
DRB         350
TOV         350
PF          350
TOT_MP      350
PER         350
TS%         350
3PAr        350
FTr         350
ORB%        350
DRB%        350
TRB%        350
AST%        350
STL%        350
BLK%        350
TOV%        350
USG%        350
OWS         350
DWS         350
OBPM        350
DBPM        350
BPM         350
VORP        350
PRE_ODDS    350
dtype: int64

In [42]:
#grabs raptor rankings
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-raptor/historical_RAPTOR_by_player.csv"
raptor_df = pd.read_csv(url)
raptor_df = raptor_df[['player_name', 'season', 'raptor_offense', 'raptor_defense', 'raptor_total']]
raptor_df.rename(columns={'player_name' : 'Player', 'season' : 'Year'}, inplace=True)
raptor_df['Year'] = raptor_df['Year'].astype(float)
raptor_df['Player'] = raptor_df['Player'].str.strip()
df = df.merge(raptor_df, how='inner', on=['Player','Year'])

In [58]:
#gives 4 players errors
temp = df.merge(saved_df, how='outer', on='Player', indicator=True)
temp = temp[temp['_merge'] != 'both']
temp

,Rank_x,Year_x,MVP_x,Player,Age_x,Tm_x,First_x,Pts Won_x,Pts Max_x,Share_x,G_x,MP_x,PTS_x,TRB_x,AST_x,STL_x,BLK_x,FG%_x,FT%_x,WS_x,WS/48_x,Rk_x,Pos_x,GS_x,FG_x,FGA_x,3P_x,3PA_x,3P%_x,2P_x,2PA_x,2P%_x,eFG%_x,FT_x,FTA_x,ORB_x,DRB_x,TOV_x,PF_x,TOT_MP_x,PER_x,TS%_x,3PAr_x,FTr_x,ORB%_x,DRB%_x,TRB%_x,AST%_x,STL%_x,BLK%_x,TOV%_x,USG%_x,OWS_x,DWS_x,OBPM_x,DBPM_x,BPM_x,VORP_x,PRE_ODDS_x,Rank_y,Year_y,MVP_y,Age_y,Tm_y,First_y,Pts Won_y,Pts Max_y,Share_y,G_y,MP_y,PTS_y,TRB_y,AST_y,STL_y,BLK_y,FG%_y,FT%_y,WS_y,WS/48_y,Rk_y,Pos_y,GS_y,FG_y,FGA_y,3P_y,3PA_y,3P%_y,2P_y,2PA_y,2P%_y,eFG%_y,FT_y,FTA_y,ORB_y,DRB_y,TOV_y,PF_y,TOT_MP_y,PER_y,TS%_y,3PAr_y,FTr_y,ORB%_y,DRB%_y,TRB%_y,AST%_y,STL%_y,BLK%_y,TOV%_y,USG%_y,OWS_y,DWS_y,OBPM_y,DBPM_y,BPM_y,VORP_y,PRE_ODDS_y,_merge
2400,NaN,NaN,NaN,Peja Stojaković,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2004,0,26.0,SAC,1.0,281.0,1230,0.228,81.0,40.3,24.2,6.3,2.1,1.3,0.2,0.480,0.927,13.5,0.198,375,SF,81,8.2,17.1,3.0,6.8,.433,5.2,10.3,.511,.566,4.9,5.2,1.1,5.1,1.9,2.0,3264,21.8,.624,.400,.307,3.2,14.1,8.8,9.1,1.7,0.3,8.9,23.9,11.4,2.1,5.2,-1.3,3.9,4.9,0,right_only
2401,NaN,NaN,NaN,Manu Ginóbili,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,2008,0,30.0,SAS,0.0,9.0,1260,0.007,74.0,31.1,19.5,4.8,4.5,1.5,0.4,0.460,0.860,11.1,0.232,152,SG,23,6.1,13.3,2.1,5.3,0.401,4.0,8.0,.499,.540,5.1,6.0,0.9,3.9,2.7,2.3,2299,24.3,.612,.395,.449,3.3,14.7,9.1,26.4,2.6,1.1,14.5,28.7,6.6,4.5,5.6,2.7,8.3,6.0,0,right_only
2402,NaN,NaN,NaN,Manu Ginóbili,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,2011,0,33.0,SAS,0.0,20.0,1210,0.017,80.0,30.3,17.4,3.7,4.9,1.5,0.4,0.433,0.871,9.9,0.195,166,SG,79,5.5,12.7,1.9,5.5,0.349,3.6,7.2,.497,.509,4.5,5.1,0.5,3.2,2.2,2.0,2426,21.7,.581,.433,.403,2.1,11.8,7.1,26.4,2.6,0.8,12.7,26.0,6.6,3.2,4.1,1.3,5.4,4.5,0,right_only
2403,NaN,NaN,NaN,Nikola Jokić,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2019,0,23.0,DEN,0.0,212.0,1010,0.210,80.0,31.3,20.1,10.8,7.3,1.4,0.7,0.511,0.821,11.8,0.226,269,C,80,7.7,15.1,1.0,3.4,.307,6.7,11.7,.569,.545,3.6,4.4,2.9,8.0,3.1,2.9,2504,26.3,.589,.224,.292,9.8,27.6,18.7,37.0,2.1,1.9,15.4,27.4,7.5,4.3,6.2,2.9,9.1,7.0,0,right_only


In [60]:
df.to_csv('csv/historical_mvp_data.csv')